In [1]:
# Code to parallel process OTAR json file to extract sentences and entities
# (c) EMBL-EBI, June 2019
#
# Started: 7 June 2019
# Updated: 29 January  2020

_author_ = 'Santosh Tirunagari'

import os
import pandas as pd
import re
from tqdm import tqdm

import multiprocessing
import csv

import glob

# if __name__ == '__main__':
#     pool = multiprocessing.Pool(processes=15)
#     pool.map(process_each_split_to_extract_sentences, file_path)
#     pool.close()
#     pool.join()
#     print('done')

In [ ]:
def process_each_split_to_extract_sentences(complete_file_path):

    colNames = ['pmcid', 'section', 't_start', 't_end', 'd_start', 'd_end', 'text']
    OTAR_df = pd.read_csv(complete_file_path, names=colNames, encoding='utf-8')
    f_name = complete_file_path.split('/')[-1]
    
    with open(result_folder + f_name+ '.csv', 'w', newline='\n') as f1:
        public_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')


        for index_, each_annotation in enumerate(OTAR_df.itertuples()): #tqdm(enumerate(OTAR_df.itertuples()),total=len(OTAR_df)):
#         if index_ <10:
            pmc_id = each_annotation.pmcid.replace('http://europepmc.org/','')
            section = each_annotation.section
            t_start = each_annotation.t_start
            t_end = each_annotation.t_end
            d_start = each_annotation.d_start
            d_end = each_annotation.d_end
            otar_sentence = each_annotation.text
            GP = otar_sentence[t_start:t_end+1]
            DS = otar_sentence[d_start:d_end+1]

            try:
                GP.encode('latin_1').decode('utf-8').strip()  # Each char is a Unicode codepoint.
            except:
                re.sub(r'[^\x00-\x7f]', r'', GP).strip()
                
            try:
                DS.encode('latin_1').decode('utf-8').strip()  # Each char is a Unicode codepoint.
            except:
                re.sub(r'[^\x00-\x7f]', r'', DS).strip()    


            new_row = [pmc_id, section, otar_sentence, GP,DS]
            public_writer.writerows([new_row])



In [ ]:
file_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_dumps/CSV_dumps/' #Toy
result_folder = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Data/'

split_list = sorted(glob.glob(os.path.join(file_path, '*split*'))) 

for each_file in tqdm(split_list, total = len(split_list)):
    process_each_split_to_extract_sentences(each_file)


In [3]:
# Merge the GPs and DS for the same OTAR sentence

def merge_GP_DS_in_sentences(complete_file_path):
    
#     GP_final_result_folder = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Final_data/GP/'
#     DS_final_result_folder = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Final_data/DS/'
    
    final_result_folder = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Final_data/'
    colNames = ['pmcid', 'section', 'sentence', 'GP', 'DS']
    
    OTAR_df = pd.read_csv(complete_file_path, names=colNames, encoding='utf-8', sep='\t')
    f_name = complete_file_path.split('/')[-1]
    
 
    GP_df = OTAR_df['GP'].groupby([OTAR_df.pmcid, OTAR_df.section, OTAR_df.sentence]).apply(list).reset_index()
#     GP_df.to_csv(GP_final_result_folder+f_name+, sep='\t')
    
    DS_df = OTAR_df['DS'].groupby([OTAR_df.pmcid, OTAR_df.section, OTAR_df.sentence]).apply(list).reset_index()
#     DS_df.to_csv(DS_final_result_folder+f_name+, sep='\t')
    
    source_df = pd.merge(GP_df, DS_df[['sentence','DS']], on='sentence')
    df = source_df.drop_duplicates(subset=['sentence'])
    
    df.to_csv(final_result_folder+f_name, sep='\t')

In [4]:
intermediate_file_path = '/nfs/gns/literature/Santosh_Tirunagari/OTAR_Results/ML-NER-Analysis/Intermediate_data/'

split_list = sorted(glob.glob(os.path.join(intermediate_file_path, '*split*'))) 

for each_file in tqdm(split_list, total = len(split_list)):
    merge_GP_DS_in_sentences(each_file)

100%|██████████| 200/200 [36:14<00:00, 10.87s/it]


In [ ]:
# GP_df = OTAR_df['GP'].groupby([OTAR_df.pmcid, OTAR_df.section, OTAR_df.sentence]).apply(list).reset_index()
# DS_df = OTAR_df['DS'].groupby([OTAR_df.pmcid, OTAR_df.section, OTAR_df.sentence]).apply(list).reset_index()
# df = pd.merge(GP_df, DS_df[['sentence','DS']], on='sentence')